In [5]:
#필요한 라이브러리를 불러봅시다.
#sys, os는 부모 디렉토리까지 파일을 가져올 수 있도록 해줍니다.
#numpy는 행렬 계산을 편리하게 해줍니다.
#PIL은 Mnist데이터들을 간단하게 그려줍니다.
#dataset.mnist는 mnist데이터를 불러옵니다.
#matplotlib은 그래프를 편리하게 그릴 수 있게 해줍니다.

import sys, os
sys.path.append(os.pardir)
import numpy as np
from PIL import Image
from mnist import load_mnist
import matplotlib.pyplot as plt

In [6]:
#이미지 확인 및 저장을 위한 간단한 코드입니다.
def img_show(img) : 
	img = Image.fromarray(np.uint8(img))
	img.show()
def img_save(img, name) :
	img = Image.fromarray(np.uint8(img))
	img.save(name + ".png", "PNG")

In [7]:
#이번 실습에서는 계단함수로 Relu를 사용할 것입니다.
#0 이하의 값들은 모두 0으로 처리하여 전파합니다.
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [8]:
#각 Layer를 표현하는 부분입니다. Weight 행렬 W, bias의 b, 그리고 input x로 이루어져 있습니다.
#dw, db는 역전파 때 기울기를 저장하기 위함입니다.
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [9]:
#Dataset을 가져옵니다.  인터넷에 연결되어 있어야 하며, 몇 분까지 걸릴 수 있습니다.
#x는 28*28픽셀로 표현된 0~9까지의 흑백 숫자 이미지입니다.
#t는 10차원 벡터로써, 현재 이미지가 몇을 나타내는지를 나타냅니다.
#예를 들어, [0,1,0,0,0,0,0,0,0] 라면 이 이미지는 1임을 나타냅니다.0-9 순서입니다.

#normalize : 0-255까지의 숫자로 밝기를 표시하지만, 이를 정규화시켜 0-1로 변환합니다.
#flatten : 28*28의 형태인 이미지를 일렬로 펴 784*1로 만들어 줍니다.
#one_hot_label : 특정 객체의 정보를 목록으로 만든 후, 0과 1로 나타내는 것을 뜻합니다.
#t의 형태가 대표적인 one_hot_label의 예시입니다.
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, flatten =True, one_hot_label = True)

In [10]:
#다양한 세팅값들입니다.
#iters_num : 학습을 몇 번 반복할 지 표시합니다.
#train_size : training set의 크기입니다.
#batch_size : batch의 크기입니다. batch란 한 번에 학습시킬 양 정도로 생각하시면 되겠습니다.
#learning rate : 학습속도입니다. 너무 빠르면 엉뚱한 곳을 겉돌며 영원히 답을 찾을 수 없을 수도 있습니다.
#반대로 너무 작으면 답을 찾는데 너무 오래 걸릴 것입니다. cost function 부문을 검색해보세요.
#batch_mask : 전체 training set중에 어느 것을 batch로 사용할 지 결정합니다.
#weight_init_std : Weight들을 초기화할 때 쓸 변수입니다.
#input layer는 28픽셀(784), hidden layer는 50, output layer는 다시 784로 설정합니다.

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.00003
batch_mask = batch_mask = np.random.choice(train_size, batch_size)
weight_init_std = 0.01
input_size = x_train[0].shape[0]
hidden_size = 50
output_size = input_size 

In [11]:
#각 레이어의 Weight들과 bias를 초기화 해봅시다.
#W는 0-1 사이의 값을 무작위로 선정한 후, weight_init_std를 곱해줍니다.
#b는 0으로 설정합니다.

params={}
params['W1'] = weight_init_std * np.random.randn(input_size , hidden_size)
params['b1'] = np.zeros(hidden_size)
params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
params['b2'] = np.zeros(output_size)

In [12]:
#위의 값들을 가지고 layer를 만들어봅시다.
#hidden layer는 Relu를 포함합니다.
layers = {}
layers['Affine1'] = Affine(params['W1'], params['b1'])
layers['Relu'] = Relu()
layers['Affine2'] = Affine(params['W2'], params['b2'])

In [15]:
#이제 학습을 시켜봅시다. batch사이즈만큼의 인풋을 설정하고, 이를 하나씩 전파시킵니다.
#전파가 완료되면, 역전파를 하며 기울기를 구합니다.
#이후 각 기울기만큼 w와 b를 업데이트합니다.
#매 1000번째 학습시마다 현재의 loss를 Mean squared Error로 출력해봅니다.
#loss는 결과적으로 점점 줄어듭니다.

for i in range(iters_num) : 
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = np.copy(x_train[batch_mask])
    t_batch = np.copy(x_train[batch_mask])
    grads = {}
    #forward
    a1 = x_batch
    z1 = layers['Affine1'].forward(a1)
    a2 = layers['Relu'].forward(z1)
    z2 = layers['Affine2'].forward(a2)
    #get loss by the output - original input
    loss = z2 - t_batch
    #backpropagation
    dout = loss
    dout=layers['Affine2'].backward(dout)
    dout=layers['Relu'].backward(dout)
    dout=layers['Affine1'].backward(dout)
    #update gradients
    grads['W1'] = layers['Affine1'].dW
    grads['b1'] = layers['Affine1'].db
    grads['W2'] = layers['Affine2'].dW
    grads['b2'] = layers['Affine2'].db
    #update parameters
    for key in ('W1', 'b1', 'W2', 'b2') :
        params[key] -= learning_rate * grads[key]
    #check the losses
    if (i % 1000 == 0):
        print(i)
        print(np.sum(loss**2) / batch_size)

0
9.06142662356
1000
9.60793543987
2000
9.07231848014
3000
8.63376693756
4000
9.29801333683
5000
9.7553046386
6000
10.3642240275
7000
9.24153216889
8000
9.09657230767
9000
9.27201365493


In [ ]:
#이제 완성된autoencoder를 실행해봅시다.
#training set의 1~10번째 사진을 input으로 넣어보고, output과 원본을 비교해봅시다.
#저장하지 않고 즉석해서 보고 싶다면, img_show()를 이용하면 됩니다.
for k in range(11):
    a1 = x_train[k]
    z1 = layers['Affine1'].forward(a1)
    a2 = layers['Relu'].forward(z1)
    z2 = layers['Affine2'].forward(a2)	
    z2 = z2.reshape(28,28)
    z2 = z2*256
    a1 = (a1*256).reshape(28,28)
    img_save(a1, "input" + str(k))
    img_save(z2, "output" + str(k))
